## Import relevant packages

In [ ]:
# Import relevant packages 
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.naive_bayes import GaussianNB

from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

## Load the data

In [ ]:
data = pd.read_csv('data/train.csv')
print("Full train dataset shape is {}".format(data.shape))

## Explore the data

In [ ]:
data.head(5)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
plot_df = data.Transported.value_counts()
plot_df.plot(kind="bar")

In [ ]:
fig, ax = plt.subplots(5,1,  figsize=(10, 10))
plt.subplots_adjust(top = 2)

sns.histplot(data['Age'], color='b', bins=50, ax=ax[0]);
sns.histplot(data['FoodCourt'], color='b', bins=50, ax=ax[1]);
sns.histplot(data['ShoppingMall'], color='b', bins=50, ax=ax[2]);
sns.histplot(data['Spa'], color='b', bins=50, ax=ax[3]);
sns.histplot(data['VRDeck'], color='b', bins=50, ax=ax[4]);

In [ ]:
data.boxplot()
plt.xticks(rotation=45, ha='right')
plt.show()

## Clean the dataset

In [ ]:
# separates the target from the data 
y = data.Transported
X=data.iloc[:,0:13]

In [ ]:
# PassengerID and Name are not necessary for training
X = X.drop(['PassengerId', 'Name'], axis=1)

In [ ]:
# Split cabin into three separates variables 
X[["Deck", "Cabin_num", "Side"]] = X["Cabin"].str.split("/", expand=True)
X = X.drop('Cabin', axis=1)

In [ ]:
# changing the categorical data to numerical values 

class_labels = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Cabin_num', 'Side']

for cl in class_labels:
    class_le = LabelEncoder()
    Y_le = class_le.fit_transform(X[cl].values)
    X[cl] = Y_le

y = LabelEncoder().fit_transform(y.values)

In [ ]:
'''
column_order = X.columns

df_categorical = X[['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']]
df_non_categorical = X.drop(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side'], axis=1)


imr = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imr = imr.fit(df_categorical.values)
imputed_data = imr.transform(df_categorical.values)
df_categorical = pd.DataFrame(imputed_data, columns=df_categorical.columns)

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df_non_categorical.values)
imputed_data = imr.transform(df_non_categorical.values)
df_non_categorical = pd.DataFrame(imputed_data, columns=df_non_categorical.columns)

combined = pd.concat([df_categorical, df_non_categorical], axis=1)
X = combined[column_order]
'''

In [ ]:
# Check for NaN values
print('Column number of missing values')
for c in X.columns:
    n_NaN = X[c].isnull().sum()
    print(f'{c:32} {n_NaN}')

In [ ]:
# There NaN values in several of the categories which needs to be removed

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(X.values)
imputed_data = imr.transform(X.values)

X = pd.DataFrame(imputed_data, columns=X.columns)

In [ ]:
# Scale the entire dataset 
X_scaled = (X - np.mean(X, axis=0)) / np.std(X)

In [ ]:
X_scaled.boxplot()
plt.xticks(rotation=45, ha='right')
plt.show()

## Prepare the dataset for training

In [ ]:
# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6, stratify=y)

In [ ]:
# Scale the data
X_train_scale = (X_train - np.mean(X_train, axis=0)) / np.std(X_train)
X_test_scale = (X_test - np.mean(X_train, axis=0)) / np.std(X_train)

In [ ]:
X_train_scale.boxplot()
plt.xticks(rotation=45, ha='right')
plt.show()

## Train the model

#### List of classifiers 
* Perceptron
* Adaline 
* Logistic Regression
* Support vector machine
* K-nearest Neighbour 
* Decision trees
* Random forest 
* Naive Bayes 
* Gradient Boosting 

In [ ]:
pipe_svc = make_pipeline(PCA(), SVC(C=100.0, gamma=0.01, random_state=1))
pipe_svc.fit(X_train_scale, y_train)
prediction = pipe_svc.predict(X_test_scale)
print(accuracy_score(y_test, prediction))

In [ ]:
# Start the classifier
classifier = SVC()

In [ ]:
# Train the classifier
classifier.fit(X_train_scale, y_train)

In [ ]:
# Predict on the test set and calculate the accuracy
y_pred = classifier.predict(X_test_scale)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: ', accuracy)

## Load the test dataset

In [ ]:
test_data = pd.read_csv('data/test.csv')

submission_id = test_data.PassengerId

# Remove Name and passengerID
test_data = test_data.drop(['PassengerId', 'Name'], axis=1)

# Split cabin into three variables 
test_data[["Deck", "Cabin_num", "Side"]] = test_data["Cabin"].str.split("/", expand=True)
test_data = test_data.drop('Cabin', axis=1)

# Change the categorical values to numerical values
class_labels = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Cabin_num', 'Side']

for cl in class_labels:
    class_le = LabelEncoder()
    Y_le = class_le.fit_transform(test_data[cl].values)
    test_data[cl] = Y_le

# replace Nan values
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(test_data.values)
imputed_data = imr.transform(test_data.values)

test_data = pd.DataFrame(imputed_data, columns=test_data.columns)

# Scale the test data
test_data_scaled = (test_data - np.mean(X, axis=0)) / np.std(X)


In [ ]:
# Make predictions for the test dataset
submission = make_pipeline(PCA(), SVC(C=100.0, gamma=0.01, random_state=1))
submission.fit(X_scaled, y)
predictions = submission.predict(test_data_scaled)
bool_predictions = (predictions > 0.5).astype(bool)

output = pd.DataFrame({'PassengerId': submission_id,
                       'Transported': bool_predictions})

output.head()

In [ ]:
# Create a csv for Kaggle 
sample_submission_df = pd.read_csv('data/sample_submission.csv')
sample_submission_df['Transported'] = bool_predictions
sample_submission_df.to_csv('data/submission.csv', index=False)
sample_submission_df.head()